In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [2]:
root = '../working/offline/6/0'
# context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-4'
# pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-4.pooling_mask-attention_mask.grad_acc-4'
# pmodel2_name = 'deberta-v3-small.flag-pairwise14-4-cat.num_negs-4.eval.neg2'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-2'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-2.pooling_mask-attention_mask'
pmodel2_name = 'deberta-v3-small.flag-pairwise14-4-cat.num_negs-4.evalneg2.2'

In [3]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')

In [4]:
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [5]:
xp2 = gezi.load(f'{root}/{pmodel2_name}/valid.pkl')

In [6]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xp2, 'cid')

In [7]:
xc['cls_pred_ori'].shape

(424943, 100)

In [8]:
xp2['cls_pred_ori'].shape

(424943, 3)

In [9]:
ids = set(xc['id'])

In [10]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe576, a3188e54, b3f6e12d, ee7655ca, 84125b7a]"
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc326a, aa354742, 004d0eee, b9344b3d, 51709b61, 979a5b9a, af2b0426, 96c8449c]"
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e458, 465e9ac3, a1d5d434, 7b0d69c6, 8167cfe4, 8c027685, 490cee65, 24cfe6eb, 0ae3fd94, f9c995d1, e91fa12a, df9d54a6, b368450e, 2863864b, 9a829640, e6d30107, 5d2477a0, 01890991, 4a630d33, e902c3ad, 1f3749ad, a3ad710d, 7778a045, 95f37e93, 1f6fc839, a6b63513, 7cce9349, 65865e00, f8658f91, e92f292f, 489802aa, 11087190, 468d72f8, b789509e, a286a29c, 6812ae4b, 21ed7c42, e23919ef, 38b9ef54, dbd197c8, 647abaa1, 7bbcc573, 432a5f22, 9d7f5be8, ef7e6cfe, 7d149..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb7f4, de59dec8, 56c58dd6, 06fd00d9, a38c5996, c40c8f99, 60c81822, 4349c1aa, 4d1a0078, 7d32145e, 69383ee9, a7586092, d0c350c9, 98461fee, 25999581, b607b4cf, cb6a19b3, 11c8223a, 35705444, e0ff7e21, 68d8bd12, 62f32737, 90ce24a6, e2273244, 578cc8c7, b423993a, df5d3538, a44eecd0, 4e1213c3, 012dd24e, cdd2244f, 07daa78a, ab425c32]"
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501363, 94d3d43a, 9ecece20, 808d31ab, 615cd02a, 6da92b81, 568fa01a, 6603b9b9, dff53cea, 42476427, 3d63b500, bd24750b, 6306ba97, 592fa3ca, 70598ae5, 21992a54, 05915708, c6f0a7fb, 7da89797, fe931e4a, 57b14df5, ca9d112f, 9d2b623b, e0ffad5f, caa2a0e4, b6cb47da, 9d30d44b, 351cb961, cc2c96eb, 3d393ce3, ab3ae8d0, d9ae76ec, 9fed798a, 1a39fcb9, f9249928, a2f89492, 1ff43b24, dde2e9dd, 93b8809e, 3b9b6120, 6ff8d8cc, 30a931e1, 5fdb303a, fb5c88e1, 9df82070, a9e1c..."


In [11]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [12]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9003963088216335

In [13]:
df_p = pd.DataFrame(xp)

In [14]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [15]:
df_p.columns

Index(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred',
       'match_rank', 'match_code', 'n_code_cell', 'probs', 'sims'],
      dtype='object')

In [16]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [17]:
df_train = pd.read_feather('../working/train.fea')

In [18]:
df_train = df_train[df_train.id.isin(ids)]

In [19]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [20]:
gezi.set_fold(df, 5, 'ancestor_id')

In [21]:
# df.head()

In [126]:
from numba import jit
@jit(nopython=True)
def feat_speedup(arr):
    r_max, r_min, r_sum = -1e5,1e5,0
    for x in arr:
        r_max = max(r_max, x)
        r_min = min(r_min, x)
        r_sum += x
    return r_max, r_min, r_sum, r_sum/len(arr)

np_lin = np.linspace(0,1,7)

@jit(nopython=True)
def sorted_quantile(array, q):
    n = len(array)
    index = (n - 1) * q
    left = int(index)
    fraction = index - left
    right = left
    right = right + int(fraction > 0)
    i, j = array[left], array[right]
    return i + (j - i) * fraction

def get_percentile(array):
    x = np.sort(array)
    n = len(x)-1
    return x[[int(n*t) for t in np_lin[1:-1]]]


df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5, top6 = [], [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob, top6_prob = [], [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim, top6_sim = [], [], [], [], []
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
dics = []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
  # if len(idxes) > 5:
  #   top6.append((idxes[5] + 0.5) / (n_code + 1))
  #   top6_prob.append(probs[idxes[5]])
  #   top6_sim.append(sims[idxes[5]])
  # else:
  #   top6.append(-1)
  #   top6_prob.append(-1)
  #   top6_sim.append(-1)
  dic = {}
  for i,ntile in enumerate([sorted_quantile(probs,i) for i in np_lin]):
    dic[f'p_trend{i}'] = ntile
  for i,ntile in enumerate(get_percentile(probs)):
    dic[f'p_ntile{i}'] = ntile
    
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])

  for i,ntile in enumerate([sorted_quantile(probs,i) for i in np_lin]):
    dic[f'c_trend{i}'] = ntile
  for i,ntile in enumerate(get_percentile(probs)):
    dic[f'c_ntile{i}'] = ntile
  dics.append(dic)
  # ctail.append((idxes[-1] + 0.5) / FLAGS.num_classes)
  # ctail_prob.append(probs[idxes[-1]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
dic = gezi.list2batch(dics)
df['top2'] = top2
df['top2_prob'] = top2_prob
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
# df['top6'] = top6
# df['top6_prob'] = top6_prob
# df['top6_sim'] = top6_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
# df['ctail'] = ctail
# df['ctail_prob'] = ctail_prob
df['pred_p2'] = xp2['pred']
df['cls_pred_p2'] = xp2['cls_pred']
p2_max_prob = []
p2_top2 = []
p2_top2_prob = []

for i in tqdm(range(len(df)), desc='p2'):
  code_idxes = xp2['code_idxes'][i]
  n_code = df['n_code_cell'].values[i]
  probs = gezi.softmax(xp2['cls_pred_ori'][i])
  idxes = (-probs).argsort()
  p2_top2.append((code_idxes[idxes[1]] + 0.5) / (n_code + 1))
  p2_max_prob.append(probs[idxes[0]])
  p2_top2_prob.append(probs[idxes[1]])
df['p2_max_prob'] = p2_max_prob
df['p2_top2'] = p2_top2
df['p2_top2_prob'] = p2_top2_prob

df['pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['pred_diff11'] = abs(df['pred_c'] - df['pred_p2'])
df['pred_diff12'] = abs(df['reg_pred'] - df['pred_p2'])
df['pred_diff13'] = abs(df['cls_pred_c'] - df['pred_p2'])
# df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]
for key in dic:
  df[key] = dic[key]
# reg_pred_probs = []
# for row in tqdm(df.itertuples(), total=len(df), desc='cls_pred_p_prob'):
#   probs = row.probs
#   n_code = row.n_code_cell
#   reg_pred = row.reg_pred
#   try:
#     reg_pred_probs.append(probs[int(min(reg_pred, 1) * (n_code + 1) - 0.5)])
#   except Exception:
#     ic(n_code, len(probs), reg_pred)
#     break
# df['reg_pred_prob'] = reg_pred_probs

top:   0%|          | 0/424943 [00:00<?, ?it/s]

p2:   0%|          | 0/424943 [00:00<?, ?it/s]

In [127]:
df['top2s_prob'] = df['max_prob'] + df['top2_prob']
df['p2_top2s_prob'] = df['p2_max_prob'] + df['p2_top2_prob']

In [128]:
# dfg = df.groupby('id')['cls_pred_p2'].apply(list).reset_index(name='preds')
# dfg

In [129]:
# dfg['mean_cls_pred_p2'] = dfg.preds.apply(lambda x: np.asarray(x).mean())

In [130]:
# df = df.merge(dfg[['id', 'mean_cls_pred_p2']], on='id')

In [131]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [132]:
df.columns

Index(['id', 'cell_id', 'cid', 'cell_type', 'rank', 'rel_rank',
       'context_match', 'match_code', 'n_code_cell', 'n_markdown_cell',
       'n_cell', 'cls_pred_c', 'pred_c', 'reg_pred', 'cls_pred_ori',
       'match_rank', 'cls2_pred', 'pred_p', 'cls_pred_p', 'max_prob',
       'max_sim', 'probs', 'sims', 'ancestor_id', 'n_words', 'source', 'fold',
       'markdown_frac', 'span', 'top2', 'top2_prob', 'top2_sim', 'top3',
       'top3_prob', 'top3_sim', 'top4', 'top4_prob', 'top4_sim', 'top5',
       'top5_prob', 'top5_sim', 'ctop_prob', 'ctop2', 'ctop2_prob', 'ctop3',
       'ctop3_prob', 'ctop4', 'ctop4_prob', 'pred_p2', 'cls_pred_p2',
       'p2_max_prob', 'p2_top2', 'p2_top2_prob', 'pred_diff0', 'pred_diff1',
       'pred_diff2', 'pred_diff3', 'pred_diff4', 'pred_diff5', 'pred_diff6',
       'pred_diff7', 'pred_diff8', 'pred_diff9', 'pred_diff10', 'pred_diff11',
       'pred_diff12', 'pred_diff13', 'rule_pred', 'top6', 'top6_prob',
       'top6_sim', 'ctail', 'ctail_prob', 'top2s_

In [133]:
df.head()

,id,cell_id,cid,cell_type,rank,rel_rank,context_match,match_code,n_code_cell,n_markdown_cell,n_cell,cls_pred_c,pred_c,reg_pred,cls_pred_ori,match_rank,cls2_pred,pred_p,cls_pred_p,max_prob,max_sim,probs,sims,ancestor_id,n_words,source,fold,markdown_frac,span,top2,top2_prob,top2_sim,top3,top3_prob,top3_sim,top4,top4_prob,top4_sim,top5,top5_prob,top5_sim,ctop_prob,ctop2,ctop2_prob,ctop3,ctop3_prob,ctop4,ctop4_prob,pred_p2,cls_pred_p2,p2_max_prob,p2_top2,p2_top2_prob,pred_diff0,pred_diff1,pred_diff2,pred_diff3,pred_diff4,pred_diff5,pred_diff6,pred_diff7,pred_diff8,pred_diff9,pred_diff10,pred_diff11,pred_diff12,pred_diff13,rule_pred,top6,top6_prob,top6_sim,ctail,ctail_prob,top2s_prob,p2_top2s_prob,p_trend0,p_trend1,p_trend2,p_trend3,p_trend4,p_trend5,p_trend6,p_ntile0,p_ntile1,p_ntile2,p_ntile3,p_ntile4,c_trend0,c_trend1,c_trend2,c_trend3,c_trend4,c_trend5,c_trend6,c_ntile0,c_ntile1,c_ntile2,c_ntile3,c_ntile4
0,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,markdown,0,0.0556,1,0.0000,8,1,9,0.0604,0.0599,0.0594,"[-4.0903583, 0.5349037, 1.3602955, 4.452827, 5.0728393, 12.519949, 1.5408992, 4.9013124, 4.9309707, 1.6340737, 1.8126483, 0.37320215, 3.3962739, 3.456965, 6.0820775, -0.9351299, 8.320439, -0.9596569, 3.6772232, 0.94654, 0.69394386, -1.5883194, 0.2900058, -0.53428674, -0.5877164, 2.7730763, -2.8129313, 6.2470136, -5.390407, 0.9643065, -0.5121496, -0.20953551, -5.1841574, -4.3066177, -5.1355834, 0.06634861, -1.7293935, 2.8716495, 7.3404927, -5.2138805, 1.8496656, -0.21122411, -0.61255383, -2.4...",0,0.0550,0.0556,0.0667,0.9425,0.7760,"[0.9425, 0.043493357, 0.0003159318, 0.0062844837, 0.0023759853, 0.0036886416, 0.00038924295, 0.00046750862, 0.00048478023]","[0.7759645, 0.71513796, 0.6177492, 0.6768827, 0.65764815, 0.6663461, 0.6218758, 0.6254989, 0.6262163]",272b483a,49,"Hi there,\n\nIs it ok that the same measurement have different target labels between signals?\nAccording to data description it should be the same (or not really?) . There are 38 cases of measurements with not consistent labels between signals.\n\nQuick and dirty code to show the problem below:",2,0.1111,0.1111,0.1667,0.0435,0.7151,0.3889,0.0063,0.6769,0.6111,0.0037,0.6663,0.5000,0.0024,0.6576,0.7849,0.3250,0.1032,0.3450,0.0425,0.3550,0.0369,0.0556,0.0613,0.9906,0.1667,0.0071,0.0043,0.0038,0.0048,0.0006,0.1068,0.1073,0.1063,0.1117,0.1111,0.3285,0.3333,0.0043,0.0038,0.0048,0.0556,0.9444,0.0005,0.6262,0.6750,0.0000,0.9860,0.9977,0.9425,0.0291,0.0043,0.0024,0.0026,0.0004,0.0005,0.0004,0.0005,0.0024,0.0037,0.0063,0.0000,0.0002,0.7849,0.0005,0.0001,0.0000,0.0000,0.0000,0.0000,0.0001,0.0002,0.0006
1,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,markdown,11,0.9545,1,10.0000,10,2,12,0.9314,0.9468,0.9621,"[-5.061675, -1.4873273, -0.1935645, 0.73971826, 3.7059398, -1.0378878, 2.9945228, -0.28360847, -4.26981, -6.2672567, -2.640204, -2.3210342, -0.064608425, 2.5773876, -5.5735707, 0.6210708, -2.8622255, -6.8944244, -6.7471614, -6.0759635, -3.0708852, -1.7026366, 1.6791137, -5.988488, -3.2226806, -3.927346, -8.143602, -7.395439, -5.5367537, -4.981709, -1.7702756, 0.15076023, -8.57975, -2.5830643, -5.663102, -7.1179194, -7.2118225, -5.558069, -5.7455144, -1.7939783, 1.1488931, -7.8734956, -1.8099...",0,0.9550,0.9545,0.9385,0.9221,0.8411,"[0.008434403, 0.0019666976, 0.0004421279, 3.3661232e-05, 0.00016652851, 3.1338517e-05, 3.9218787e-05, 0.00014034826, 0.002613414, 0.064007655, 0.92212564]","[0.7482277, 0.719436, 0.68992156, 0.6389959, 0.6706126, 0.637582, 0.6420177, 0.66723025, 0.7250581, 0.7883054, 0.8410588]",de362dd8,9,### Support vector classifier scored 0.8442 on test set,1,0.1667,0.0909,0.8636,0.0640,0.7883,0.0455,0.0084,0.7482,0.7727,0.0026,0.7251,0.1364,0.0020,0.7194,0.7849,0.3250,0.1032,0.3450,0.0425,0.3550,0.0369,0.9545,0.9381,0.8954,0.8636,0.0992,0.0078,0.0076,0.0232,0.0005,0.0831,0.0985,0.0677,0.0914,0.0909,0.8859,0.9091,0.0078,0.0076,0.0232,0.9545,0.2273,0.0004,0.6899,0.4950,0.0000,0.9861,0.9946,0.0084,0.0010,0.0001,0.0000,0.0001,0.0231,0.9221,0.0000,0.0

In [134]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'cls_pred_c',
          'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          # 'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'pred_diff11',
          'pred_diff12',
          'pred_diff13',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          'top5_prob',
          'top5_sim',
          # 'top6',
          # 'top6_prob',
          # 'top6_sim',
          'ctop_prob', 
          'ctop2', 
          'ctop3', 
          'ctop4', 
          'ctop2_prob', 
          'ctop3_prob', 
          'ctop4_prob',
          # 'ctail',
          # 'ctail_prob',
          'pred_p2',
          'cls_pred_p2',
          'p2_max_prob',
          'p2_top2',
          'p2_top2_prob',
          # 'p2_top3',
          # 'p2_top3_prob',
          # 'mean_cls_pred_p2',
          'top2s_prob',
          'p2_top2s_prob',
        ]
for key in dic:
  reg_cols += [key]
# for i in range(FLAGS.num_classes):
#   reg_cols += [f'cls_pred{i}']
cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[label_col]
X_valid = dvalid[cols]
y_valid = dvalid[label_col]

In [135]:
from catboost import CatBoostRegressor
xgb_params = {'learning_rate': 0.02,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

In [136]:
model = CatBoostRegressor(**xgb_params)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=100,
                )  

0:	learn: 0.2544051	test: 0.2544051	test1: 0.2543314	best: 0.2543314 (0)	total: 59.8ms	remaining: 9m 58s
100:	learn: 0.0827963	test: 0.0827963	test1: 0.0844345	best: 0.0844345 (100)	total: 4.82s	remaining: 7m 52s
200:	learn: 0.0654684	test: 0.0654684	test1: 0.0676945	best: 0.0676945 (200)	total: 9.52s	remaining: 7m 43s
300:	learn: 0.0633372	test: 0.0633372	test1: 0.0658133	best: 0.0658133 (300)	total: 14.3s	remaining: 7m 39s
400:	learn: 0.0624167	test: 0.0624167	test1: 0.0651437	best: 0.0651437 (400)	total: 19s	remaining: 7m 33s
500:	learn: 0.0615864	test: 0.0615864	test1: 0.0646361	best: 0.0646361 (500)	total: 23.7s	remaining: 7m 29s
600:	learn: 0.0609125	test: 0.0609125	test1: 0.0642757	best: 0.0642757 (600)	total: 28.5s	remaining: 7m 25s
700:	learn: 0.0603534	test: 0.0603534	test1: 0.0640006	best: 0.0640006 (700)	total: 33.3s	remaining: 7m 21s
800:	learn: 0.0598212	test: 0.0598212	test1: 0.0637644	best: 0.0637644 (800)	total: 38s	remaining: 7m 16s
900:	learn: 0.0593117	test: 0.05931

In [137]:
dvalid['cb_pred'] = model.predict(dvalid[cols])

In [138]:
ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

[07/25/22 08:29:28] 2876217204.py:1 in <module>
                    calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}): 0.9122073807841654


0.9122073807841654

In [139]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          'min_data_in_leaf': 300,
          # 'max_depth': 10,
          'learning_rate': 0.03,
          "feature_fraction": 0.8,
          "bagging_fraction": 0.75,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

bst = lgb.train(params,
                d_train,
                200000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=500)
dvalid['lgb_pred'] = bst.predict(dvalid[cols])
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023248 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14364
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 58
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[100]	training's l1: 0.0719021	valid_1's l1: 0.0733799
[200]	training's l1: 0.0648248	valid_1's l1: 0.0665069
[300]	training's l1: 0.0634809	valid_1's l1: 0.0652874
[400]	training's l1: 0.0625362	valid_1's l1: 0.0644406
[500]	training's l1: 0.0620676	valid_1's l1: 0.064046
[600]	training's l1: 0.0617227	valid_1's l1: 0.0637695
[700]	training's l1: 0.0615237	valid_1's l1: 0.0636183
[800]	training's l1: 0.0613164	valid_1's l1: 0.0634625
[900]	training's l1: 0.0611465	valid_1's l1: 0.0633448
[1000]	training's l1: 0.0609797	valid_1's l1: 0.063227
[1100]	training's l1: 0.0608486	valid_1's 

0.9133472549351369

In [140]:
gezi.plot.feature_importance(model, topn=20)

In [141]:
gezi.plot.feature_importance(bst, topn=20)

In [142]:
gezi.plot.feature_importance_df(model)

,feat_name,importanace
0,pred_c,9.9755
1,cls_pred_p2,9.8264
2,pred_p2,9.6576
3,reg_pred,7.2371
4,cls_pred_p,6.8412
5,cls_pred_c,6.5681
6,p2_top2,4.1642
7,pred_p,4.1035
8,cls2_pred,3.1611
9,top2,2.7598


In [143]:
gezi.plot.feature_importance_df(bst)

,feat_name,importanace
0,top2_prob,15905
1,max_prob,15708
2,markdown_frac,15156
3,p_trend0,14509
4,p_trend1,14399
5,p2_top2,14303
6,pred_diff3,13608
7,top5,13431
8,p_trend2,13375
9,p_trend3,13363


In [144]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

0.9133472549351369

In [145]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})

0.9122073807841654

In [146]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values + dvalid.lgb_pred.values) / 2.})

0.9132382089902022

In [147]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})

0.9133993801478208